In [ ]:
""" 
General import statements
These can be used throughout the notebook
"""

import crash_analysis
import pandas as pd
from matplotlib import pyplot as plt 
%matplotlib inline

In [ ]:
"""
IO
How to write/read data to/from a file.
How to extract a dataframe from a directory of zipfiles
"""

from crash_analysis import read_csv
from crash_analysis.parser import extract_zipfiles, xmldocs_to_dataframe

# If you are getting new data from zipfiles, set new_data to True. Otherwise, read from CSV
new_data = False
# location of a CSV file with cached data.
cached_data = 'src/data/2016070012_jan24_6pm.csv'
# location of zipflies, i.e. the raw crash reports. 
# Currently, an external tool is required to download all the crash reports from quickbase
zipfile_path = 'C:\\CrashReports\\'

if not new_data:
    # How to read data from CSV
    df = read_csv(cached_data)
else:
    # Get data from source XML files
    extract_zipfiles(zipfile_path)  
    df = xmldocs_to_dataframe(zipfile_path)  # maps contents of zipfiles to dataframe
    df.to_csv(cached_data, encoding='utf-8')  # caches new data
    

# Make sure to remove duplicate data!
df.drop_duplicates(inplace=True)

In [ ]:
"""
Exploring the dataframe
"""
from crash_analysis import get_columns

# Get the columns of the dataframe
columns = get_columns(df)
print(columns)

# Get columns with native Pandas syntax
columns2 = df.columns 
print(columns2)

# Read the top few entries of the dataframe
print(df.head())

# Read the last few entries of the dataframe
print(df.tail())

In [ ]:
"""
Filtering the dataframe

See http://pandas.pydata.org/pandas-docs/stable/10min.html#selection 
for more about indexing and selection.
"""

from crash_analysis import filter_dataframe, get_column

# Before filtering
print('Unfiltered number of crashes: ' + str(len(df)))

# How to filter
# filter_dataframe(df: DataFrame,  Key:Column_name=Value:Column_value)
basic_only_df = filter_dataframe(df, AppName='ProSeries Basic Edition - 2016')

# After filtering
# Notice that there are fewer crashes in this dataframe!
print('Filtered number of crashes: ' + str(len(basic_only_df)))

# Filter multiple columns
admin_network_df = filter_dataframe(df, InstallType='Network', WorkStationType='Admin')
print('Number of Network Admins: ' + str(len(admin_network_df)))

# Filter multiple values per column
specific_os = filter_dataframe(df, OperatingSystem=['Windows 10 Home Build 14393', 'Windows 10 Pro Build 14393' ])
print('Number of Windows 10 users: ' + str(len(specific_os)))

# Get a specific column
problem_descriptions_df = get_column(df, 'ProblemDescription')
print('Problem description df columns: ', problem_descriptions_df.columns)

# Native Pandas way to get a column
problem_descriptions_df2 = df['ProblemDescription']
print('Problem description df columns: ', problem_descriptions_df2.columns)

# Native Pandas way to boolean index
specific_cust_num = df[df['CustNum'] == '0614456913']
print(specific_cust_num)

In [19]:
# Reading Text Data
customer_desc_df = crash_analysis.get_column(current_version_df, 'Customer_Description')
print(customer_desc_df)

## Remove NaN values --> two options
full_desc = crash_analysis.remove_empty(customer_desc_df)
# full_desc = crash_analysis.fill_empty(customer_desc_df)
print(full_desc.head())

## Formatting Printed Output
# import pandas as pd
# pd.set_option('display.height', 5000)
# pd.set_option('display.max_rows', 5000)
# pd.set_option('display.max_colwidth', 250)

95241                                                  NaN
95305                        CONVERTIN RETURN FOR E-FILING
95309              CONVERTING RETURN FOR ELECTRONIC FILING
95312                                                  NaN
95324                           down loading state updates
95326                                      Initial Install
95327                                           signing in
95328                                                  NaN
95329                                                  NaN
95331                                                  NaN
95332                                                  NaN
95333                                  LOGIN TO MY ACCOUNT
95334    Installing updates at the same time logging in...
95335                                                  NaN
95337       Trying to complete the 2016 Proseries download
95338                              logging into my account
95339                                                  N

In [20]:
# Gathering some simple statistics

# crash_analysis.get_columns(current_version_df)
## Find histograms of field

current_version_df['Error_Code'].value_counts()[:20]

268690347    48
26581991     16
25336807     12
8559591      12
268707747    11
25992167     11
23501799     11
16030695     10
25730023     10
19635175     10
25861095     10
8100839      10
24222695     10
23567335     10
23239655      9
7052263       9
17865703      9
23960551      9
24681447      9
6134759       9
Name: Error_Code, dtype: int64

In [4]:
from crash_analysis import analysis
from crash_analysis.preprocess import tokenize_stem_stop, tokenize

analysis.stem_frequency(full_desc)
print

total words: 3176
instal     	:  222 	 [['install'], ['installing'], ['installing'], ['installing'], ['install'], ['installing']]
2016       	:  170 	 [['2016'], ['2016'], ['2016'], ['2016'], ['2016'], ['2016']]
proseri    	:  121 	 [['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries'], ['proseries']]
program    	:  110 	 [['program'], ['program'], ['program'], ['program'], ['program'], ['program']]
open       	:  103 	 [['opening'], ['openning'], ['opening'], ['opening'], ['open'], ['open']]
updat      	:   99 	 [['updates'], ['updates'], ['updating'], ['updating'], ['updates'], ['updates']]
download   	:   90 	 [['download'], ['downloading'], ['downloading'], ['downloading'], ['downloading'], ['downloading']]
sign       	:   87 	 [['signing'], ['signing'], ['signing'], ['sign'], ['signing'], ['signing']]
tri        	:   80 	 [['trying'], ['trying'], ['trying'], ['trying'], ['trying'], ['trying']]
file       	:   78 	 [['filing'], ['filing'], ['file'], ['file'], [

In [43]:
# Groupings
# print df_xml.columns
# print df_xml.groupby(['WorkStationType', 'OSIs64Bit', 'OperatingSystem'])['CrashGUID'].count()
# 
# # Group by keyterm
# from crash_analysis import analysis
# 
# analysis.associate_by_keyterms(df_xml, 'Message', 'StackTrace', print_output=True)
# print

In [23]:
from crash_analysis import analysis

from crash_analysis.preprocess import ngram, lower_and_tokenize, tokenize_stem_stop

import re


def word_trigram(text): return ngram(lower_and_tokenize(text), 5, skip=2)


def word_trigram_stemmed(text): return ngram(tokenize_stem_stop(text), 10, skip=1)


sorted_counts, total, vocab = analysis.stem_frequency(full_desc, _map=word_trigram_stemmed, print_output=False)

filter(lambda tuple: len(re.split(r'\W', tuple[0])) > 2, sorted_counts)[:20]

[(u'instal proseri 2016', 14),
 (u'instal 2016 proseri', 11),
 (u'open client file', 10),
 (u'instal proseri basic', 10),
 (u'2016 stop work', 9),
 (u'proseri basic 2016', 8),
 (u'instal pro seri', 6),
 (u'instal 2016 program', 6),
 (u'proseri basic edit', 6),
 (u'set onlin account', 5),
 (u'2016 proseri softwar', 5),
 (u'2016 proseri basic', 5),
 (u'proseri 2016 stop work', 5),
 (u'instal proseri basic edit', 5),
 (u'proseri 2016 stop', 5),
 (u'tri open partnership', 4),
 (u'download proseri 2016', 4),
 (u'proseri 2016 basic', 4),
 (u'sign first time', 4),
 (u'enter confirm code', 4)]

[(u'instal 2016', 40),
 (u'instal proseri', 32),
 (u'proseri 2016', 31),
 (u'instal updat', 30),
 (u'stop work', 29),
 (u'proseri basic', 23),
 (u'instal program', 22),
 (u'open client', 22),
 (u'2016 proseri', 22),
 (u'tri open', 19),
 (u'2016 program', 19),
 (u'instal softwar', 18),
 (u'client file', 16),
 (u'download 2016', 15),
 (u'pro seri', 15),
 (u'first time', 14),
 (u'instal proseri 2016', 14),
 (u'open file', 13),
 (u'2016 softwar', 12),
 (u'enter password', 12)]


[('trying to', 70),
 ('signing in', 39),
 ('a client', 28),
 ('proseries 2016', 28),
 ('the program', 27),
 ('installing 2016', 26),
 ('i was', 24),
 ('proseries basic', 23),
 ('setting up', 23),
 ('sign in', 23),
 ('in to', 23),
 ('installing updates', 22),
 ('installing the', 22),
 ('2016 proseries', 22),
 ('installing proseries', 22),
 ('to open', 21),
 ('logging in', 21),
 ('stopped working', 21),
 ('opening a', 21),
 ('a file', 20)]

In [23]:
def char_ngram(text): return ngram(text, 20, skip=1, delim='')


sorted_counts, total, vocab = analysis.stem_frequency(full_desc, _map=char_ngram, print_output=False)

# filter(lambda tuple: len(tuple[0].split(' ')) > 2, sorted_counts)[:50]

In [14]:
# filter(lambda tuple: len(tuple[0].split(' ')) > 4, sorted_counts)[:50]

In [ ]:
# Group by queries